In [108]:
# Setup -- from CS 231N GANs notebook
from __future__ import print_function, division
import tensorflow as tf
import numpy as np

import os
import nibabel as nib
from nibabel.testing import data_path


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'



In [4]:
# A bunch of utility functions

def show_images(images):
    images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return

def show_multimodal(image):
    # images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    figdim = int(np.ceil(image.shape[0]/2))
    # sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))
    
    dispInd = int(np.ceil(image.shape[3]/2))

    fig = plt.figure(figsize=(figdim, figdim))
    gs = gridspec.GridSpec(figdim, figdim)
    gs.update(wspace=0.05, hspace=0.05)

    for i in range(image.shape[0]):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(image[i,:,:,dispInd])
    return

def preprocess_img(x):
    return 2 * x - 1.0

def deprocess_img(x):
    return (x + 1.0) / 2.0

def rel_error(x,y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def count_params():
    """Count the number of parameters in the current TensorFlow graph """
    param_count = np.sum([np.prod(x.get_shape().as_list()) for x in tf.global_variables()])
    return param_count


def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

In [28]:
# Open 
path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_flair.nii'
flair_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_t1.nii'
t1_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_t2.nii'
t2_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_t1ce.nii'
t1ce_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_seg.nii'
seg_img = nib.load(path).get_data()

img = np.zeros((4, 240, 240, 155))
img[0,:,:,:] = flair_img
img[1,:,:,:] = t1_img
img[2,:,:,:] = t2_img
img[3,:,:,:] = t1ce_img

img = img[:,:,:,:-3]

# show_multimodal(img)
seg_out = np.zeros((5,240,240,152))
seg_img = seg_img[:,:,:-3]

for i in range(4):
    seg_out[i,:,:,:] = np.equal(seg_img,i)
    
print(seg_out.min())

0.0


In [ ]:
# # Compress files into numpy compressed form --- maybe use later
# i = 0
# for root, dirs, files in os.walk('../HGG/'):
#     if len(files) > 1:
#         HGGnp = np.zeros((4,240,240,155))

#         flair_img = nib.load(root + '/' + files[-5]).get_data()
#         HGGnp[0,:,:,:] = flair_img
        
#         seg = nib.load(root + '/' + files[-4]).get_data()
#         HGGsegnp = seg
        
#         t1_img = nib.load(root + '/' + files[-3]).get_data()
#         HGGnp[1,:,:,:] = flair_img
        
#         t1ce_img = nib.load(root + '/' + files[-2]).get_data()
#         HGGnp[2,:,:,:] = flair_img
        
#         t2_img = nib.load(root + '/' + files[-1]).get_data()
#         HGGnp[3,:,:,:] = flair_img
        
#         np.savez_compressed(root + '/' + files[-1][:-7], img = HGGnp, seg = HGGsegnp)
        
#         i +=1
#         print(i)


# # HGG = tf.Variable(tf.stack(HGGnp), name="HGG", trainable=False)

# i = 0
# for root, dirs, files in os.walk('../LGG/'):
#     if len(files) > 1:
#         LGGnp = np.zeros((4,240,240,155))

#         flair_img = nib.load(root + '/' + files[-5]).get_data()
#         LGGnp[0,:,:,:] = flair_img
        
#         seg = nib.load(root + '/' + files[-4]).get_data()
#         LGGsegnp = seg
        
#         t1_img = nib.load(root + '/' + files[-3]).get_data()
#         LGGnp[1,:,:,:] = flair_img
        
#         t1ce_img = nib.load(root + '/' + files[-2]).get_data()
#         LGGnp[2,:,:,:] = flair_img
        
#         t2_img = nib.load(root + '/' + files[-1]).get_data()
#         LGGnp[3,:,:,:] = flair_img
        
#         np.savez_compressed(root + '/' + files[-1][:-7], img = LGGnp, seg = LGGsegnp)
        
#         i +=1
#         print(i)

# # LGG = tf.Variable(tf.zeros([75,4,240,240,155]), name="LGG", trainable=False)

        
        

In [ ]:
# LGGnp = np.zeros((75,4,240,240,155))
# i = 0
# for root, dirs, files in os.walk('../LGG/'):
#     if len(files) > 1:
#         LGGnp = np.zeros((4,240,240,155))

#         flair_img = nib.load(root + '/' + files[-5]).get_data()
#         LGGnp[0,:,:,:] = flair_img
        
#         seg = nib.load(root + '/' + files[-4]).get_data()
#         LGGsegnp = seg
        
#         t1_img = nib.load(root + '/' + files[-3]).get_data()
#         LGGnp[1,:,:,:] = flair_img
        
#         t1ce_img = nib.load(root + '/' + files[-2]).get_data()
#         LGGnp[2,:,:,:] = flair_img
        
#         t2_img = nib.load(root + '/' + files[-1]).get_data()
#         LGGnp[3,:,:,:] = flair_img
        
#         np.savez_compressed(root + '/' + files[-1][:-7], img = LGGnp, seg = LGGsegnp)
        
#         i +=1
#         print(i)
        
# LGG = tf.Variable(tf.stack(LGGnp), name="LGG", trainable=False)

In [53]:
# Data feeding function

def BRATSdatafeed(batch_size):
    """
    Generate training batches for the BRATS data
    
    The image resizing is done as follows:
        - 40 slices removed from beginning and end of dims 0 and 1
        - 4 removed from start and 7 removed from end of dim 2
        - Image futher downsampled via mean pooling at beginning of CNN
    
    """
    
    i = 0
    j = 0
    inds = np.random.randint(1, 256, batch_size)
    
    inData = np.zeros((batch_size,160,160,144,4))
    labelData = np.zeros((batch_size,160,160,144,5))
    
    for root, dirs, files in os.walk('../HGG/'):
        if (j>=batch_size):
            break
        if (i < 190) and (len(files) > 1) and (i==inds[j]): 
            flair_img = nib.load(root + '/' + files[-5]).get_data()
            inData[j,:,:,:,0] = flair_img[40:-40,40:-40,4:-7]

            t1_img = nib.load(root + '/' + files[-3]).get_data()
            inData[j,:,:,:,1] = t1_img[40:-40,40:-40,4:-7]

            t1ce_img = nib.load(root + '/' + files[-2]).get_data()
            inData[j,:,:,:,2] = t1ce_img[40:-40,40:-40,4:-7]

            t2_img = nib.load(root + '/' + files[-1]).get_data()
            inData[j,:,:,:,3] = t2_img[40:-40,40:-40,4:-7]

            seg = nib.load(root + '/' + files[-4]).get_data()
            seg_img = seg
            
            for i in range(5):
                labelData[j,:,:,:,i] = np.equal(seg_img[40:-40,40:-40,4:-7],i)
            
            j+= 1
            
        
        i +=1
        
        
    for root, dirs, files in os.walk('../LGG/'):
        if (j>=batch_size):
            break
        if (len(files) > 1) and (i==inds[j]): 
            flair_img = nib.load(root + '/' + files[-5]).get_data()
            inData[j,:,:,:,0] = flair_img[40:-40,40:-40,4:-7]

            t1_img = nib.load(root + '/' + files[-3]).get_data()
            inData[j,:,:,:,1] = t1_img[40:-40,40:-40,4:-7]

            t1ce_img = nib.load(root + '/' + files[-2]).get_data()
            inData[j,:,:,:,2] = t1ce_img[40:-40,40:-40,4:-7]

            t2_img = nib.load(root + '/' + files[-1]).get_data()
            inData[j,:,:,:,3] = t2_img[40:-40,40:-40,4:-7]

            seg = nib.load(root + '/' + files[-4]).get_data()
            seg_img = seg
            
            for i in range(5):
                labelData[j,:,:,:,i] = np.equal(seg_img[40:-40,40:-40,4:-7],i)
            
            j+= 1
            
        
        i +=1
    
    x = inData#.reshape([-1,160*160*144*4])
    y = labelData.reshape([-1,160*160*144*5])
#     x = tf.convert_to_tensor(inData)
#     x = tf.reshape(x,[-1,240*240*152*4])
#     y = tf.convert_to_tensor(labelData)
#     y = tf.reshape(y,[-1,240*240*152*5])
    
    return x, y



In [54]:
# Test data feed function
x, y = BRATSdatafeed(20)
print(y.shape)

(20, 18432000)


In [55]:
# Dice Loss
def DICEscore(yFull, prediction):
    #
    # truth, estimate = tf tensors of size N x voxels*channels
    # Computes average Dice scores
    #
    # loss = scalar tensor containing Dice loss 
    # 
    
    
            
    score1 = 2*tf.reduce_mean(tf.divide(tf.reduce_sum(tf.multiply(truth,estimate),axis=1),
                       (tf.reduce_sum(truth,axis=1) + tf.reduce_sum(estimate,axis=1))))
    return loss

In [155]:
def brainconvnet(x):
    """Funciton to run the computational graph
    
    Inputs:
    - x: TensorFlow Tensor of flattened input images, shape [batch_size, #voxels * 4]
    IMPORTANT: 
        -x must be a channels-first tensor (before resizing)
        -remove BOTTOM three slices of x before passing in --- bottom three layers will not contain
            much information and this makes sizing much easier within the network
    
    Returns:
    TensorFlow Tensor with shape [batch_size, #voxels * 5]
    (Graph calculates 5th order tensor: channel 1 for image, channel 2 for class, others for voxels)
    """
    with tf.variable_scope("convNet") as scope:
        input_size = x.get_shape().as_list()
#         x_reshape = tf.reshape(x, [-1,160,160,144,4])
#         print(input_size)
        x_downsample = tf.layers.average_pooling3d(inputs = x, pool_size = (2,2,2),
                                        strides = (2,2,2), padding='valid',name=None)
#         print(x_downsample.get_shape())
        conv1 = tf.layers.conv3d(inputs=x_downsample, filters=8, 
                                 kernel_size=[5, 5, 5],padding="same", activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling3d(inputs = conv1, pool_size = (2,2,2),
                                        strides = (2,2,2), padding='valid',name=None)
#         print(pool1.get_shape())
        conv2 = tf.layers.conv3d(inputs=pool1, filters=8, 
                                 kernel_size=[5, 5, 5],padding="same", activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling3d(inputs = conv2, pool_size = (2,2,2),
                                        strides = (2,2,2), padding='valid',name=None)
#         print(pool2.get_shape())
        conv3 = tf.layers.conv3d(inputs=pool2, filters=32, 
                                 kernel_size=[3, 3, 3],padding="same", activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling3d(inputs = conv3, pool_size = (2,2,2),
                                        strides = (2,2,2), padding='valid',name=None)
#         print(pool3.get_shape())
        conv4 = tf.layers.conv3d(inputs=pool3, filters=128, 
                                 kernel_size=[3, 3, 3],padding="same", activation=tf.nn.relu)
#         print(conv4.get_shape())
#         return conv4
            
    with tf.variable_scope("deconvNet") as scope:
        W4 = tf.Variable(tf.truncated_normal([3, 3, 3, 16, 128], stddev=0.1))
        deconv4 = tf.nn.conv3d_transpose(conv4, filter = W4, output_shape = [input_size[0],20, 20, 18, 16], 
                                         strides = [1,2,2,2,1])
#         deconv4 = tf.layers.conv3d_transpose(conv4, filters = 16, kernel_size = [3,3,3], strides = (2,2,2))
        b4 = tf.Variable(tf.constant(0.1, shape=[16]))
        relu4 = tf.nn.relu(deconv4 + b4)
#         print(relu4.get_shape())
        
        W3 = tf.Variable(tf.truncated_normal([3, 3, 3, 8, 16], stddev=0.1))
        deconv3 = tf.nn.conv3d_transpose(relu4, filter = W3, output_shape = [input_size[0],40, 40, 36, 8], 
                                         strides = [1,2,2,2,1])
        b3 = tf.Variable(tf.constant(0.1, shape=[8]))
        relu3 = tf.nn.relu(deconv3 + b3)
#         print(relu3.get_shape())
        
        W2 = tf.Variable(tf.truncated_normal([3, 3, 3, 8, 8], stddev=0.1))
        deconv2 = tf.nn.conv3d_transpose(relu3, filter = W2, output_shape = [input_size[0],80, 80, 72, 8], 
                                         strides = [1,2,2,2,1])
        b2 = tf.Variable(tf.constant(0.1, shape=[8]))
        relu2 = tf.nn.relu(deconv2 + b2)
#         print(relu2.get_shape())
        
        W1 = tf.Variable(tf.truncated_normal([3, 3, 3, 5, 8], stddev=0.1))
        deconv1 = tf.nn.conv3d_transpose(relu2, filter = W1, output_shape = [input_size[0],160, 160, 144, 5], 
                                         strides = [1,2,2,2,1])
        b1 = tf.Variable(tf.constant(0.1, shape=[5]))
        img = tf.reshape(deconv1 + b1,[input_size[0],5*160*160*144])
#         print(img.get_shape())
        
        return img 

In [156]:
# Get number of network parameters
def paramcount():
    
    tf.reset_default_graph()
    with get_session() as sess:
        y = brainconvnet(tf.ones((2, 160,160,144,4)))
        cur_count = count_params()
        return cur_count

print(paramcount())

191277


In [158]:
# Build graph for the network
tf.reset_default_graph()

with get_session() as sess:
    sess.run(tf.global_variables_initializer())
    
    inputPlaceholder = tf.placeholder(tf.float32, shape = [5, 160,160,144,4])
    outputPlaceholder = tf.placeholder(tf.float32, shape = [5, 160*160*144*5])
    x, y = BRATSdatafeed(5)
#     print(x.shape)
    
    logits = brainconvnet(inputPlaceholder)
    segLoss = tf.reduce_mean(tf.losses.softmax_cross_entropy(outputPlaceholder , logits))
    segSolve = tf.train.AdamOptimizer(learning_rate = 1e-3, beta1 = 0.5).minimize(segLoss)
    
    feed = {inputPlaceholder: x, outputPlaceholder: y}
    
    loss, _ = sess.run([segLoss, segSolve],
                            feed_dict=feed)
        
    


# batch_size = 20

# placeholders for images from the training dataset
# batch_size = tf.placeholder(tf.int32)


# segvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'brainconvnet')




# correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




FailedPreconditionError: Attempting to use uninitialized value convNet/conv3d/kernel
	 [[Node: convNet/conv3d/kernel/read = Identity[T=DT_FLOAT, _class=["loc:@convNet/conv3d/kernel"], _device="/job:localhost/replica:0/task:0/cpu:0"](convNet/conv3d/kernel)]]

Caused by op u'convNet/conv3d/kernel/read', defined at:
  File "/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(Z

In [144]:
# Run model
losses = []
bdices = []

numEpoch = 1
batch_size = 5
numBatches = np.int(np.ceil(256 / batch_size * numEpoch))


for batch in range(numBatches):
    x, y = BRATSdatafeed(batch_size)
    feed = {inputPlaceholder: x, outputPlaceholder: y}

    pred, loss = sess.run(segSolve,feed_dict=feed)
    losses.append(loss)
    bdice = DICEscore(y, pred)
    bdices.append(bdice)
    print("DICE Score: %s, Loss: %s" % (bdice,loss))



RuntimeError: Attempted to use a closed Session.